# **TP Cours Textmining** 
#### *Etude portée sur les tweets de deux politiques français*

*05-02-2021*

Les objectifs de ce TP sont : 
- Travailler sur du texte français
- Analyser les données 
- Reprendre les acquis développés dans les TP précédents
- Découvrir de nouveaux outils : scattertext
- Prédire qui a posté un tweet 

## **1. Installation des packages**

In [1]:
# !pip install scattertext
# !pip install spacy
# !pip install nltk
# !pip install termcolor

     |████████████████████████████████| 7.3 MB 4.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 24.0 MB 7.3 MB/s             
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9309 sha256=787abd3c1097f668f15953a507739eb9b4140ed3fe3390973f846c4a1bb047af
  Stored in directory: /Users/greg/Library/Caches/pip/wheels/8d/62/8b/71813348245ae1bcbae179193bbc72db819e8057e89298a6ac
Successfully built flashtext
You should consider upgrading via the '/Users/greg/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/greg/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/greg/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/greg/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
#!python -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6 MB 3.7 MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-py3-none-any.whl size=90338488 sha256=9062b02022a5efc7bb41052b6b2a9e1753867d3ef32b529ca9e5440633ab5fd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ooskq1qu/wheels/2e/26/ff/ce93eb966e7176ebe81e6c98209582e13e108cdd2d6d636df0
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import time
import datetime

# Modules de traitement du texte
import spacy
import fr_core_news_md
import nltk
import re
from termcolor import colored

# Modules pour le wordcloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Module pour scattertext
import scattertext as st

# Modules de modélisation
from sklearn.utils.fixes import loguniform
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay, classification_report


In [3]:
# chemin où se trouve le jeu de données (tweets_politics_2022.csv)
PATH_DATA = '/data' #TODO mettre le chemin pour trouver le fichier # 

## **2. Prise en main de la base de données**

Les données ont été extraites via l'API tweepy dans un autre notebook. \
Les tweets de certains candidats à l'élection présidentiels ont été récupérés. 

Regardez les variables à disposition, quelques comptages, s'il y a des données manquantes, quelques graphiques (?), la spécificité des tweets, etc. 

#### Import des données

In [5]:
df_tweets = pd.read_csv(f'tweets_politics_2022.csv')

In [6]:
df_tweets.shape

(18430, 6)

In [7]:
df_tweets.head()

,id,created_at,favorite_count,retweet_count,text,user_id
0,1487771447992926210,2022-01-30 12:55:29,436.0,193.0,Redonner du sens à la gauche : se rassembler a...,JeanLuc_Melenchon
1,1487400906517848070,2022-01-29 12:23:05,2350.0,1027.0,L’inscription sur les listes électorales c’est...,JeanLuc_Melenchon
2,1487117751084855300,2022-01-28 17:37:55,1145.0,480.0,🔴 Rendez-vous ce dimanche 30 janvier à 20h55 s...,JeanLuc_Melenchon
3,1487104482336989191,2022-01-28 16:45:12,1164.0,437.0,3 solutions pour financer la retraite à 60 ans...,JeanLuc_Melenchon
4,1487080530558476288,2022-01-28 15:10:01,1591.0,551.0,La Méditerranée est le plus grand cimetière du...,JeanLuc_Melenchon


####  Quelques comptages / graphiques

##### Indicateurs simples sur les variables : 
- Y a't'il des données manquantes ? 
- combien de tweets de chaque candidat ? 
- dates minimales / maximales des tweets
- Distribution des favoris et des retweets de chaque candidat

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
    <li> Utiliser .isnull() </li>
    <li><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.value_counts.html" >pd.value_counts()</a> pour compter le nombre de modalités d'une variable catégorielle </li>
    <li> Convertir la date au bon format avec pd.to_datetime() </li>
    <li> Vous pouvez utiliser la fonction groupby et describe() </li>

</ul>
</p>

In [ ]:
# Implémentez la fonction en changeant le None
def check_missing_values(df):
  #TODO
  return None

check_missing_values(df_tweets)

In [ ]:
# Combien de tweets dans la base de données pour chacun des candidats ? 
#TODO

In [ ]:
# A quelles dates ont été envoyés les premiers / derniers tweets des candidats ? 
#TODO

In [ ]:
# Quelle est la distribution des favoris et retweets des candidats  ?
#TODO

##### Répartition du nombre de retweets / favoris dans le temps 

In [14]:
def visualize_count_favorites(df, userID) : 
  
  ''' Cette fonction permet de visualiser le nombre de favoris et de retweets 
  sur toute la période pour un user_id donné '''

  df_temp = df.loc[df["user_id"] == userID]
  ylabels = ["favorite_count", "retweet_count"]

  print("Représentation des nombres de retweets et de favoris de chaque tweet de {} par date".format(userID))
  fig = plt.figure(figsize=(13,3))
  fig.subplots_adjust(hspace=0.01,wspace=0.01)

  n_row = len(ylabels)
  n_col = 1
  for count, ylabel in enumerate(ylabels):
      ax = fig.add_subplot(n_row, n_col, count + 1)
      ax.plot(df_temp["created_at"], df_temp[ylabel])
      ax.set_ylabel(ylabel)
  
  plt.show()

In [1]:
visualize_count_favorites(df_tweets, "JeanLuc_Melenchon")
print("\n")
visualize_count_favorites(df_tweets, "Marine_Lepen")

> **Question** : Qu'observe-t'on ? 

**Réponse** : TODO

##### Taille des tweets par politique 

Est-ce que des candidats font des tweets + ou - longs que d'autres ? 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
    <li> Utilisez la fonction <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html" >.split()</a> pour calculer la taille des tweets</li>
    <li> Vous pouvez utiliser la fonction groupby et describe() </li>

</ul>
</p>

In [ ]:
# Calcul d'une variable contenant le nombre de mots de chaque tweets
#TODO

# Calcul de la distribution de la variable pour chaque politique
#TODO

**Réponse** : TODO

##### Lecture de quelques tweets

In [16]:
def print_famous_tweets(userID, nb_favorites) :

  ''' Cette fonction permet de sélectionner les tweets qui ont eu le plus de favoris 
  pour un user_id donné, et de lire le tweet avec les indicateurs des autres variables de la 
  base de données  
  '''

  df_sub = df_tweets.loc[(df_tweets.user_id==userID) & (df_tweets.favorite_count > nb_favorites),:]
  for irow in range(df_sub.shape[0]):
      df_row = df_sub.iloc[irow,:]
    
      print(df_row["created_at"])
      print("favorite_count={:6} retweet_count={:6}".format(df_row["favorite_count"],df_row["retweet_count"]))
      print(colored(df_row["text"], 'magenta'))
      print("\n")

In [ ]:
print_famous_tweets("JeanLuc_Melenchon", 20000)

In [ ]:
print_famous_tweets("Marine_Lepen", 10000)

In [ ]:
print_famous_tweets("Eric_Zemmour", 20000)

> **Question** : Qu'y-a't'il de particulier dans les tweets par rapport à un texte normal ?

**Réponse** : TODO

### **Filtres**

- Filtre sur la date pour ne prendre en compte que la campagne électorale (début septembre 2021)
- Filtre sur certains candidats pour que les traitements ne soient pas trop longs

In [ ]:
DATE_MIN = "2021-09-01 00:00:00"

df_tweets = df_tweets.loc[df_tweets["created_at"] >= datetime.datetime.strptime(DATE_MIN, "%Y-%m-%d %H:%M:%S")] 

print(f"Taille du dataframe : {len(df_tweets)}")

In [ ]:
candidats_select = [] #TODO : choisir des candidats
                    
df_tweets = df_tweets.loc[df_tweets.user_id.isin(candidats_select)]

print(f"Taille du dataframe : {len(df_tweets)}")

## **3. Preprocessing du texte**

On va prendre en compte les particularités des tweets pour nettoyer le texte. \
On va tester les techniques de preprocessing des cours précédents sur du texte français : 
- stopwords
- lemmatisation
- tokenisation


### Nettoyage du texte
Dans cette partie du TP, on nettoie le texte pour enlever les mots qui vont rajouter du bruit à l'analyse (et ne rien apporter) \
Pour nettoyer le texte : 
- suppression des chiffres
- suppression de certaines expressions grâce à des expressions régulières
- suppression des stopwords


In [ ]:
# on charge le modèle français de spacy
nlp = fr_core_news_md.load()
print(len(nlp.Defaults.stop_words))

# on peut rajouter des stopwords à la liste de spacy de cette manière : 
nlp.Defaults.stop_words |= {"avoir", "falloir", "faire", "monsieur", "direct",
                            "interview", "livetweet", "suivez", r"invité\w+", r"(chaîne )?youtube", "mlp"}

# boucle pour que les stopwords ajoutés fonctionnent
for word in nlp.Defaults.stop_words :
    for w in (word, word[0].capitalize(), word.upper()):
        lex = nlp.vocab[w]
        lex.is_stop = True

# nombre de stopwords 
len(nlp.Defaults.stop_words)

> **Conseil** :  Regarder toujours la liste entière de stopwords proposés pour enlever certains mots qui seraient utiles dans votre étude ou rajouter des stopwords non présents dans la liste

La cellule ci-dessous donne un exemple d'informations que peut donner Spacy : 

In [ ]:
doc = nlp("Demain je travaille \\n\\n à la maison. #fatigué @hetik \\n https://test.com")

list_spacy = []
                
for token in doc : 
  list_spacy.append([token.text,
                        token.idx,
                        token.lemma_,
                        token.is_punct,
                        token.is_space,
                        token.is_alpha,
                        token.shape_,
                        token.pos_,
                        token.tag_,
                        token.ent_type_])
  
exemple_spacy = pd.DataFrame(list_spacy, columns=["text", "idx","lemma","is_punct","is_space","is_alpha","shape","pos","tag","ent_type"])
exemple_spacy

Expressions régulières pour nettoyer le texte 

In [ ]:
regexp_link = re.compile(r"http\S+") # suppression des liens
regexp_number = re.compile(r"\d+[h., ]?\d*") # suppression des chiffres

**TODO** : Créer une expression régulière pour supprimer les hashtags et @ 

Remplacer le #TODO dans la cellule suivante par une expression régulière.

Votre regexp fonctionne si vous trouver " ça  marche  !!"

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
Lorsque vous cherchez à créer des expressions régulières, vous pouvez vous aider en allant sur ce site : <a href="https://regex101.com/" >regex101.com</a> 
</p> 

In [ ]:
# suppression des hashtags et @
regexp_hashtags = re.compile(r"#TODO")    

test_hashtags = "#Fuck ça #ne marche @pas !!"
re.sub(regexp_hashtags, "", test_hashtags)

Création de la fonction de nettoyage du texte 

**TODO** : coder plusieurs fonctions :      
- une fonction `clean_text_spacy` qui prend en entrée un tweet et utilise spacy pour :     
    - supprimer les ponctuations ; 
    - supprimer les stopwords ; 
    - supprimer les caractères de type espace (/n, /t, etc.)
Cette fonction garde les tokens entiers
- une fonction `clean_lemmatize` :     
    - supprimer les ponctuations ; 
    - supprimer les stopwords ; 
    - supprimer les caractères de type espace (/n, /t, etc.)
Cette fonction garde non pas les tokens entiers, mais les lemmes. 
- une fonction chapeau `preprocess_tweet` qui : 
  - met les mots en minuscule
  - supprime les mots des expressions régulières
  - au choix applique la fonction `clean_text_spacy` ou `clean_lemmatize`

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
Lorsque vous utilisez les fonctions de spacy, vous allez potentiellement les tokeniser directement (et récupérer une liste au lieu d'un texte). Pour éviter cela, transformez le résultat de cette manière :    

```
result = " ".join(result)
```

</p> 

In [ ]:
def clean_txt_spacy(doc):
  txt = #TODO
  return txt

def clean_lemmatize(doc):
  lemmatized_txt = #TODO
  return lemmatized_txt


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
- Utiliser re.sub() pour supprimer les liens, hashtags, chiffres
</p> 

In [ ]:
def preprocess_tweet(text, lemmatizing = True):

  '''Fonction permettant de nettoyer le texte. Elle renvoie un string (pas de tokenisation encore)'''
  text_clean = text.lower().encode('utf-8').decode('utf-8')
  #TODO : supprimer du texte les liens, hashtags et chiffres avec les regexp précédentes
  text_clean = #TODO suppression liens
  text_clean = #TODO suppression hashtags
  text_clean = #TODO suppression chiffres

  doc = nlp(text_clean)  
  if lemmatizing : 
    preprocessed_tweet = clean_lemmatize(doc)
  else : 
    preprocessed_tweet = clean_txt_spacy(doc)

  return preprocessed_tweet

In [ ]:
# exemple pour tester sa fonction 
tweet_test = "Ils Pensaient se moquer #non, ils m'ont donné 1 slogan !😄 \n\n- Entretien à découvrir et partager \n\nhttps://t.co/Yn60Areagu"
preprocess_tweet(tweet_test, lemmatizing=True)

Réponse attendue : 
```python 
'pensaient moquer donner slogan 😄 entretien découvrir partager'
```

In [ ]:
# On peut alors nettoyer nos tweets, et créer une nouvelle colonne, text_preprocess
# cela peut prendre un peu de temps à tourner
df_tweets["text_preprocess"] = df_tweets["text"].apply(lambda tweet : preprocess_tweet(tweet, lemmatizing=True))

In [ ]:
# On regarde le résultat du nettoyage du texte
pd.set_option("max_colwidth", None)
df_tweets[["text", "text_preprocess"]].head(10)

> Le preprocess n'est pas encore parfait, on pourrait enlever les verbes avec du pos-tagging ou bien rajouter l'info de pos-tagging après chaque mot. \
> Supprimer les emojis ou les transformer en texte.

### Tokenisation
On tokenise la colonne de tweets prétraités (preprocess)

**TODO** : utiliser le module nltk pour tokeniser un tweet avec la fonction tokenisation

In [ ]:
nltk.download('punkt') # nécessaire pour la tokenisation

In [ ]:
# Remplir le None dans le code
def tokenisation(tweet):
  tweet_tokenized = #TODO
  return(tweet_tokenized)

In [ ]:
df_tweets["tokens"] = df_tweets["text_preprocess"].apply(lambda tweet : tokenisation(tweet))

In [ ]:
df_tweets[["text_preprocess", "tokens"]].head()

### Analyse du preprocess

On regarde un peu les résultats du preprocessing : 
- combien y a-t-il de mots distincts pour chacun des deux hommes politiques ? 
- Quels sont les mots les plus utilisés par deux candidats de votre choix ? 

Pour cela vous vous aiderez des deux fonctions données ci-dessous

In [ ]:
def create_big_tweet_by_userid(userid, col_text) : 

  ''' Fonction pour mettre tous les tweets de chaque politiciens dans un même text (string) '''
  one_big_tweet = " ".join(df_tweets.loc[df_tweets["user_id"] == userid, col_text])
  
  return one_big_tweet
  

In [ ]:
def get_n_most_common_words(list_words, n) :

  ''' Fonction permettant de donner les n mots les plus fréquents d'une liste de mots '''
  freq_words = nltk.FreqDist(list_words)
  print(freq_words.most_common(n))


**TODO** : Si on n'utilise pas de preprocessing, quels sont les mots les plus utilisés par les 2 politiciens ?

In [ ]:
# Créer un gros tweet pour chacun des deux politiques (qui est la jointure de l'ensemble de ses tweets)
big_tweet_candidate1 = #TODO
big_tweet_candidate2 = #TODO

# Tokeniser le gros tweet de chacun des politiques
tokens_candidate1 = #TODO
tokens_candidate2 = #TODO

# Regarder les 10 mots les plus communs pour chacun des politiques
#TODO pour chaque candidat

**Réponse** : 

Sans preprocessing, combien y a-t-il de mots distincts pour chaque politique ?

In [1]:
# la fonction set appliquée sur une liste donne une liste d'éléments uniques
print("Nombre de mots distincts dans les tweets du candidat 1 : {}".format(len(set(tokens_candidate1))))
print("Nombre de mots distincts dans les tweets du candidat 2 : {}".format(len(set(tokens_candidate2))))

**Réponse** : 

Jean Luc Mélenchon : 11877 \
Eric Zemmour : 8960 \
Marine Lepen : 8108 \
Emmanuel Macron : 4394

**TODO** : même question avec un preprocessing ?


In [ ]:
# Créer un gros tweet pour chacun des deux politiques (qui est la jointure de l'ensemble de ses tweets)
big_tweet_candidate1 = #TODO
big_tweet_candidate2 = #TODO

# Tokeniser le gros tweet de chacun des politiques
tokens_candidate1 = #TODO
tokens_candidate2 = #TODO

# Regarder les 10 mots les plus communs pour chacun des politiques
#TODO pour chaque candidat

In [ ]:
print("Nombre de mots distincts dans les tweets du candidat 1 : {}".format(len(set(tokens_candidate1))))
print("Nombre de mots distincts dans les tweets du candidat 2 : {}".format(len(set(tokens_candidate2))))

**Réponse** : 

Jean Luc Mélenchon : 5369 \
Eric Zemmour : 4628

### Nuage de mots

On trace un nuage de mots pour chacun des politiques pour voir ce qui ressort

**#TODO** : Faire un nuage de mots pour deux candidats de votre choix avec 30 mots

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
    <li> transformer l'ensemble des tweets d'un politique en un texte unique </li>
    <li> vous pouvez utiliser la fonction WordCloud </li>
</ul>
</p>

In [ ]:
# Faire un texte unique pour les tweets de JLM 
#TODO
print("Wordcloud des mots lemmatisés de l'ensemble des tweets du candidat 1")
#Faire le wordcloud 
#TODO

Wordcloud des mots lemmatisés de l'ensemble des tweets du candidat 1


In [ ]:
# Faire un texte unique pour les tweets de JLM 
#TODO
print("Wordcloud des mots lemmatisés de l'ensemble des tweets du candidat 2")
#Faire le wordcloud 
#TODO

Wordcloud des mots lemmatisés de l'ensemble des tweets du candidat 2


C'est bien beau, mais c'est difficile à analyser, et surtout à comparer... \
On va utiliser scattertext pour comparer réellement le vocabulaire des 2 politiques.

## **4. Scattertext**

Grâce à Scattertext, on va pouvoir comparer de manière visuelle la distinction de vocabulaire utilisé par deux candidats de votre choix. 


On doit d'abord construire un corpus avec nos données : 
- donner la variable de catégorie 
- donner la variable du texte

On peut rajouter le partie ```.compact(st.AssociationCompactor(4000))``` pour ne prendre en compte que les 4000 mots les plus importants dans le scattertext.

**TODO** : créer le corpus avec la fonction donnée ci-dessous

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
    <li> Filtrer en gardant les tweets des deux candidats de votre choix </li>
</ul>
</p>

In [ ]:
df_sample = df_tweets.loc[df_tweets.user_id.isin([#TODO : candidats de votre choix])]

# on crée un objet corpus pour scattertext
corpus = st.CorpusFromPandas(data_frame = df_sample,
                             category_col = , #TODO
                             text_col = , #TODO
                             nlp = nlp).build().compact(st.AssociationCompactor(4000))

Une fois le corpus créé, on peut créer le html avec le scattertext. 

On utilise la fonction ```st.produce_scattertext_explorer``` en donnant les paramètres vus pendant le cours : 
- term_ranker
- term_scorer
- transform 

**TODO** : remplir la fonction en réfléchissant aux paramètres que vous voulez tester.

In [ ]:
# On crée le html du scattertext
html = st.produce_scattertext_explorer(  corpus
                                       , category                  = #TODO
                                       , category_name             = #TODO
                                       , not_category_name         = #TODO
                                       , minimum_term_frequency    = #TODO
                                       , pmi_threshold_coefficient = 1
                                       , term_ranker               = #TODO
                                       , term_scorer               = #TODO
                                       , transform                 = #TODO
                                       , width_in_pixels           = 1000
                                       )

# On enregistre le html
open("tweets_visualisation.html", 'wb').write(html.encode('utf-8'))

1449245

**TODO** : regarder le résultat (il apparaitra normalement dans le dossier content du notebook) en téléchargeant le html (cela peut prendre un petit moment avant de s'afficher correctement).

**Question** : Analysez le graphique

## **5. Modélisation**

On souhaite prédire si un tweet provient du compte de Marine Le Pen, de Jean Luc Mélenchon, d'Eric Zemmour ou d'Emmanuel Macron. Pour cela, on a besoin de : 
- Créer un échantillon train / dev
- préparer le text (préprocessing)
- créer des features (plusieurs méthodes : bag of words, counts of words, etc.)
- réaliser l'algorithme
- évaluer la performance du modèle 

### Création des échantillons 

**TODO** : créer un échantillon train (70% du jeu de données total) et un échantillon test 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
  <li> Utilisez la fonction <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html" >train_test_split</a></li>
  <li> Vous pouvez prendre tout le df_tweet dans le X</li>
  <li> N'oubliez pas de rajouter un random_seed pour avoir des résultats reproductibles</li>
</p> 

In [ ]:
df_train, df_test, y_train, y_test = # None

In [ ]:
print(f"Nombre de tweets dans l'échantillon train : {len(df_train)}")
print(f"Nombre de tweets dans l'échantillon test : {len(df_test)}")

**Réponse** : 

Nombre de tweets dans l'échantillon train : 4449 \
Nombre de tweets dans l'échantillon test : 1907

### Modèle de régression multinomiale sans gridsearch 

- Transformer le texte de df_train et df_test en vecteurs pour le modèle
- Utiliser la régression logistique multinomiale sans paramètre
- Regarder les paramètres sélectionnés
- Regarder le score sur l'échantillon test

**#TODO** : transformer df_train pour que ce ne soit plus des tweets, mais des vecteurs grâce à <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html" >TfidfVectorizer</a> 

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>

La fonction TfidfVectorizer a des paramètres que vous pouvez choisir : 
<ul>

- Combien de n-grams : vous considérez mot par mot ou bien également des groupes de 2 mots
- max_df : si vous voulez enlever un pourcentage de mots les plus fréquents
- min_df : si vous voulez enlever un pourcentage de mots les moins fréquents
</p> 

In [ ]:
#TODO

**#TODO** : créer le modèle de régression logistique (OVR) et entrainer le modèle sur les données d'apprentissage

In [ ]:
# initialiser le modèle 
model = #TODO

# entrainer le modèle avec les données d'apprentissage
model_default_fit = #TODO

In [ ]:
# vous pouvez voir les paramètres du modèle 
model_default_fit.get_params(deep=True)

**#TODO** : Regarder la performance du modèle sur l'échantillon train et test (accuracy)

<details>    
<summary>
    <font size="3"**texte en gras** color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
  <li> N'oubliez pas de transformer le texte de l'échantillon test en vecteur en amont</li>
  <li> Vous pouvez utiliser la fonction <a href="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">score</a>  du modèle</li>
  
</p> 

In [ ]:
#TODO
X_test = vectorizer.transform(X_test)
model_default_fit.score(X_test, y_test)

**Question** : Quel est l'accuracy sur l'échantillon test ? 

**Résultat** : **#TODO**

**#TODO** : regarder plus en détail ce que donne le modèle : 

In [ ]:
print("le 1er tweet de l'échantillon test a été prédit : ")
print(model_default_fit.predict(X_test[0]))

In [ ]:
model_default_fit.predict_proba(X_test[0])

**Question** : que retourne la ligne de code ci-dessus ?

**Réponse** : 

### Mise en place de la RandomSearch

On veut mettre en place une randomSearch pour sélectionner les meilleurs paramètres qu'on a choisi d'évaluer via la méthode de cross-validation : 
- on établit d'abord la grille de paramètres que l'on veut tester
- on effectue la RandomSearch
- on regarde les résultats sut l'échantillon test

In [10]:
# Paramètres à tester données, vous pouvez modifier pour tester d'autres paramètres
dict_params = dict(prep__text_preprocess__max_df=[0.99, 0.95, 0.9],
                   prep__text_preprocess__min_df=[2, 5, 10],
                   clf__C = [1, 20, 50],
                   clf__penalty = ['l2'],
                   clf__multi_class=['ovr', 'multinomial'])

**#TODO** : Entrainer la Randomsearch avec les paramètres ci-dessus sur les données d'apprentissage avec de la cross validation

NB : une pipeline a été mise en place dans la cellule ci-dessous afin de pouvoir tester également les paramètres du TFIDF. 

<details>    
<summary>
    <font size="3"**texte en gras** color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
  <li> Vous pouvez utiliser la fonction <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html">RandomizedSearchCV</a></li>
  
</p> 

In [ ]:
# Entrainer le randomizedsearch 

# Vectorisation de la variable text_preprocess
text_transformer_tfidf =  TfidfVectorizer()
preprocess = ColumnTransformer([("text_preprocess", text_transformer_tfidf, "text_preprocess")], 
                               remainder="drop")

# Type de modèle à tester
model = LogisticRegression(random_state=54269, max_iter=1000)

# Pipeline qui combine le preprocess et le modèle
prep_model = Pipeline(steps=[('prep',preprocess),
                             ('clf', model)])

# RANDOMIZED SEARCH
##### A MODIFIER A PARTIR D'ICI #####
random_search = RandomizedSearchCV()#TODO
### FIN MODIFICATION ### 

best_rd_model = random_search.fit(df_train, y_train)

In [ ]:
# Meilleurs paramètres sélectionnés par la randomSearch
best_rd_model.best_estimator_

Quelle accuracy le meilleur modèle a-t'il atteint sur l'échantillon train ? 

In [ ]:
# Résultats du meilleur modèle sur l'échantillon train 
best_rd_model.best_score_

**#TODO** : on évalue la performance en calculant l'accuracy du modèle sélectionné par randomsearch sur l'échantillon test



In [ ]:
grid_search.score(#TODO)

### Evaluation de la performance du modèle 

On va calculer la matrice de confusion sur l'échantillon test

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
  <li> Prédire les candidats de df_test dans un premier temps </li>
  <li> Vous pouvez utiliser la fonction <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html" >ConfusionMatrixDisplay</a></li>
  
</p> 

In [ ]:
#matrice de confusion
#confrontation entre Y observé sur l’éch. test et la prédiction
#TODO

**#TODO** : Interpréter les résultats 

**Question** : Sur l'ensemble des tweets de JLM, combien (pourcentage) ont bien été prédits JLM ?  

**Réponse** :

### Test sur des nouvelles données :

Ces quelques tweets ont été récupérés après que la base de données ait été récupérée. Ce sont donc des nouvelles données que le modèle n'a jamais vu.

**#TODO** : Qui a publié ces tweets ? 

In [ ]:
# Visualisation des tweets à prédire
df_mystere = pd.read_excel(f"{PATH_DATA}/test_mystere.xlsx")
df_mystere["text"]

In [ ]:
# On prépare les données pour que df_mystere ait la même structure que df_train
df_mystere["text_preprocess"] = df_mystere.text.apply(lambda row : preprocess_tweet(row, lemmatizing=True))
df_mystere["tokens"] = df_mystere.text_preprocess.apply(lambda row : tokenisation(row))

In [ ]:
# Réaliser la prédiction avec l'un des deux modèles réalisés
#TODO 

**Réponse attendue** : 

```
array(['Eric_Zemmour', 'Eric_Zemmour', 'JeanLuc_Melenchon',
       'Emmanuel_Macron', 'JeanLuc_Melenchon'], dtype=object)
```